In [1]:
import os
import sys
import time
import random
import time
import numpy as np
import cv2	
import threading
import carla
from queue import Queue
from queue import Empty
 
IM_WIDTH = 640
IM_HEIIGHT = 480 
start_flag = 1
 
def process_img(image,camera_queue):
	i = np.array(image.raw_data)
	i2 = i.reshape((IM_HEIIGHT,IM_WIDTH,4))
	i3 = i2[:,:,:3]
	cv2.imshow("Image",i3)
	cv2.waitKey(25)
	cv2.imwrite(os.path.join('../outputs/output_synchronized', '%06d.png' % image.frame),i3)
	camera_queue.put(image.frame)
	return i3/255.0

# def timer(seconds):
#     print("定时器已启动！")
#     threading.Timer(seconds, timeout).start()

# def timeout():
# 	start_flag = 0
#     print("定时器时间到！")


sensor_list =  []
actor_list = []

try:
	#create client
	client = carla.Client('localhost', 2000)
	client.set_timeout(10.0)
	#save original_settings
	world =  client.get_world()
	original_settings = world.get_settings()

	# set to sync mode
	settings = world.get_settings()
	settings.fixed_delta_seconds = 0.05
	settings.synchronous_mode = True
	world.apply_settings(settings)

	traffic_manager = client.get_trafficmanager()
	traffic_manager.set_synchronous_mode(True)

	# data queue
	camera_queue = Queue()

	#get blueprint libarary
	blueprint_library = world.get_blueprint_library()
	#Choose a vehicle blueprint which name is model3 and select the first one
	bp = blueprint_library.filter("model3")[0]
	print(bp)
	#Returns a list of recommended spawning points and random choice one from it
	spawn_point = world.get_map().get_spawn_points()[1]
	#spawn vehicle to the world by spawn_actor method
	vehicle = world.spawn_actor(bp,spawn_point)
	#control the vehicle
	vehicle.set_autopilot(enabled=True)

	#add vehicle to the actor list
	actor_list.append(vehicle)

	# add camera
	cam_bp = blueprint_library.find("sensor.camera.rgb")
	#set the attribute of camera
	cam_bp.set_attribute("image_size_x",f"{IM_WIDTH}")
	cam_bp.set_attribute("image_size_y",f"{IM_HEIIGHT}")
	cam_bp.set_attribute("fov","110")
	#add camera sensor to the vehicle
	spawn_point = carla.Transform(carla.Location(x=2.5,z=0.7))
	sensor = world.spawn_actor(cam_bp,spawn_point,attach_to=vehicle)

	actor_list.append(sensor)
	sensor_list.append(sensor)

	sensor.listen(lambda image: process_img(image, camera_queue))
	spectator = world.get_spectator()

	n = 0
	while(True):
		world.tick()
		loc = vehicle.get_transform().location
		spectator.set_transform(carla.Transform(carla.Location(x=loc.x,y=loc.y,z=35),carla.Rotation(yaw=0,pitch=-90,roll=0)))
		if(n > 200):
			break
		n = n+1

		# 使用queue.get()函数，在数据处理完成之前，阻止程序的推进
		try:
			for i in range(0, len(sensor_list)):
				s_frame = camera_queue.get(True, 1.0)
				print("    Frame: %d " % (s_frame))
		except Empty:
			print("   Some of the sensor information is missed")
finally:
	world.apply_settings(original_settings)
	for actor in actor_list:
		actor.destroy()
	print("All cleaned up!")

ActorBlueprint(id=vehicle.tesla.model3,tags=[vehicle, tesla, model3])
    Frame: 30444 
    Frame: 30445 
    Frame: 30446 
    Frame: 30447 
    Frame: 30448 
    Frame: 30449 
    Frame: 30450 
    Frame: 30451 
    Frame: 30452 
    Frame: 30453 
    Frame: 30454 
    Frame: 30455 
    Frame: 30456 
    Frame: 30457 
    Frame: 30458 
    Frame: 30459 
    Frame: 30460 
    Frame: 30461 
    Frame: 30462 
    Frame: 30463 
    Frame: 30464 
    Frame: 30465 
    Frame: 30466 
    Frame: 30467 
    Frame: 30468 
    Frame: 30469 
    Frame: 30470 
    Frame: 30471 
    Frame: 30472 
    Frame: 30473 
    Frame: 30474 
    Frame: 30475 
    Frame: 30476 
    Frame: 30477 
    Frame: 30478 
    Frame: 30479 
    Frame: 30480 
    Frame: 30481 
    Frame: 30482 
    Frame: 30483 
    Frame: 30484 
    Frame: 30485 
    Frame: 30486 
    Frame: 30487 
    Frame: 30488 
    Frame: 30489 
    Frame: 30490 
    Frame: 30491 
    Frame: 30492 
    Frame: 30493 
    Frame: 30494 
    Frame: 3

In [5]:
client = carla.Client('localhost', 2000)
client.set_timeout(10.0)
#save original_settings
world =  client.get_world()
for actor in actor_list:
		actor.destroy()